$$
\Large{
\text{Пошук мінімуму функції (метод оберненої матриці #1.128):}
}
$$

$$
\Large{
F_{\text{min}} = 6x_1 + 4x_2
}
$$

$$
\Large{
\text{Обмеження:}
}
$$
\begin{align*}
2x_1 + x_2 &\geq 3 \\
x_1 - x_2 &\leq 1 \\
-x_1 + 2x_2 &\geq 1 \\
x_1, x_2 &\geq 0
\end{align*}

### Зведемо до розширеної (канонічної) системи, до задачі максимуму, додамо балансуючі змінні (змінні рівності):

In [1]:
import numpy as np
np.seterr(divide='ignore')


mtx_vars_origin = [[2, 1],
                   [1, -1],
                   [-1, 2]]

mtx_vars_balance = [[-1, 0],
                    [0, 0],
                    [0, -1]]

mtx_vars_base = [[1, 0, 0],
                 [0, 1, 0],
                 [0, 0, 1]] # ця змінна генерується автоматично

Cb = [-10000, 0, -10000] # треба врахувати балансування

b = np.array([3, 1, 1])
C = np.array([-6, -4, 0, 0])

mtx_vars_all = np.concatenate((mtx_vars_origin, mtx_vars_balance), axis=1)
mtx_vars_all

array([[ 2,  1, -1,  0],
       [ 1, -1,  0,  0],
       [-1,  2,  0, -1]])

### Клас з логікою

In [2]:
class LpInvMtxStep:
    def __init__(self, A: np.array, C: np.array, b: np.array, B=None, Cb=None, current_basis_indexes=None, step=None):
        # Кількість обмежень відповідно до рівнянь у матриці А
        self.const_count = len(A)

        # Номер ітерації
        self.step = 1 if step is None else step

        # Початкова базисна матриця
        if B is not None:
            assert self.const_count == len(B)
            assert len(B) == len(B[0])
            self.B = B
        else:
            self.B = np.eye(self.const_count)

        self.vals_amount = len(A[0]) + len(self.B[0]) # кількість змінних
        self.vals_dict = {ind: f"X_{ind+1}" for ind in range(self.vals_amount)} # словник відповідності індексу - назві змінної
        if current_basis_indexes is None:
            self.current_basis_indexes = np.arange(len(A[0]), len(A[0]) + len(self.B[0])) # Змінні початкового базису
        else:
            assert len(current_basis_indexes) == self.const_count
            self.current_basis_indexes = current_basis_indexes

        str_basis_indexex = [f'X_{x+1}' for x in self.current_basis_indexes]
        self.str_basis_indexex = ', '.join(str_basis_indexex)

        # Кількість обмежень з розмірності матриці
        self.const_count = len(A[0])

        # Матриця коєфіцієнтів обмежень
        self.A = A

        # Інвертована матриця поточного базису
        self.B_inv = np.linalg.inv(self.B)

        # Значення C_b (в разі відсутності ініціюючі нулі)
        self.Cb = Cb if Cb is not None else np.zeros(len(self.B[0]))

        # Ц.ф.
        self.C = C

        # Межі обмежень
        self.b = b

        # Розрахунок алгоритму
        self.CbB_inv = np.dot(self.Cb, self.B_inv)
        self.BinvA = np.dot(self.B_inv, self.A)
        self.Binv_b = np.dot(self.B_inv, self.b)
        self.CbB_inv_b = np.dot(self.CbB_inv, self.b)

        # Визначення наступного базису та статусу припинення ітерацій (відсутність нульових значень у плані)
        self.CbBinvA_minus_C = np.dot(self.Cb, self.BinvA) - self.C
        self.stop_iter = np.all(self.CbBinvA_minus_C >= 0)
        if not self.stop_iter:
            self.min_CbBinvA_minus_C, self.new_basis_index = self.find_min_value(self.CbBinvA_minus_C)
        else:
            self.min_CbBinvA_minus_C, self.new_basis_index = None, None

        # Визначення базису на вихід
        self.min_elem_order = np.argmin(self.CbBinvA_minus_C)
        self.ratio = self.Binv_b / self.BinvA[:, self.min_elem_order]
        positive_elements = self.ratio[self.ratio > 0]
        min_positive_index = np.argmin(positive_elements)
        min_index = np.where(self.ratio == positive_elements[min_positive_index])[0][0]
        self.min_ratio_order = min_index
        self.min_ratio_val = self.ratio[min_index]

        # Формування набору даних для наступної ітерції:
        if not self.stop_iter:
            self.new_Cb = self.replace_value(self.Cb, self.min_ratio_order,self.C[self.new_basis_index])
            self.new_B = self.replace_column(self.B, self.min_ratio_order, self.A[:, self.new_basis_index])
            self.new_basis_index_set = self.replace_value(self.current_basis_indexes, self.min_ratio_order, self.new_basis_index)
        else:
            self.new_Cb, self.new_B, self.new_basis_index_set = None, None, None


    def get_result(self) -> dict:
        if self.stop_iter:
            new_data = None
        else:
            new_data = {'A': self.A,
                        'C': self.C,
                        'b': self.b,
                        'B': self.B,
                        'Cb': self.new_Cb,
                        'current_basis_indexes': self.new_basis_index_set,
                        'step': self.step + 1}

        return {'stop_status': self.stop_iter,
                'result': self.CbBinvA_minus_C,
                'iter_details': self.__str__(),
                'next_step_data': new_data}

    @staticmethod
    def replace_value(vector, index, new_value):
        vector_copy = np.copy(vector)
        vector_copy[index] = new_value
        return vector_copy

    @staticmethod
    def replace_column(matrix, column_index, vector):
        matrix_copy = np.copy(matrix)
        matrix_copy[:, column_index] = vector
        return matrix_copy

    @staticmethod
    def find_min_value(vector):
        min_value = np.min(vector)
        min_index = np.argmin(vector)
        return min_value, min_index

    def get_next_step(self):
        return {'A': self.A,
                'C': self.C,
                'b': self.b,
                'B': self.new_B,
                'Cb': self.new_Cb,
                'current_basis_indexes': self.new_basis_index_set,
                'step': self.step + 1}

    def __str__(self):
        if not self.stop_iter:
            res_str = "PLANE STILL CONSIST NEGATIVE VALUE(S)\n------------------------------"
        else:
            x_res_str = ""
            for name, value_x in zip(self.str_basis_indexex.split(', '), self.Binv_b):
                x_res_str += f"{name}: {value_x}\n"
            res_str = f"------------------------------\n" \
                      f"ITERATING STOPPED: \n" \
                      f"BEST FUNC: {self.CbB_inv_b} \n" \
                      f"VARS:      \n{x_res_str}\n" \
                      f"------------------------------"

        return f"Iter #:\n{self.step}\n\n" \
               f"Values amount:\n{self.vals_amount}\n\n" \
               f"Values dict:\n{self.vals_dict}\n\n" \
               f"Matrix A:\n{self.A}\n\n" \
               f"C:\n{self.C}\n\n" \
               f"b:\n{self.b}\n\n" \
               f"B:\n{self.B}\n\n" \
               f"Basis index:\n{self.current_basis_indexes}\n\n" \
               f"Basis varnames:\n{self.str_basis_indexex}\n\n" \
               f"B_inv:\n{self.B_inv}\n\n" \
               f"CbB_inv:\n{self.CbB_inv}\n\n" \
               f"BinvA:\n{self.BinvA}\n\n" \
               f"Binv_b:\n{self.Binv_b}\n\n" \
               f"CbB_inv_b:\n{self.CbB_inv_b}\n\n" \
               f"CbBinvA_minus_C:\n{self.CbBinvA_minus_C}\n\n" \
               f"min_elem_order:\nX{self.min_elem_order+1} = {self.CbBinvA_minus_C[self.min_elem_order]}\n\n" \
               f"ratio:\n{self.ratio}\n\n" \
               f"min ratio value:\n{self.min_ratio_val}\n\n" \
               f"leave:\n{self.vals_dict[self.current_basis_indexes[self.min_ratio_order]]}\n\n"\
               f"New Cb:\n{self.new_Cb}\n\n" \
               f"New B:\n{self.new_B}\n\n" \
               f"Result:\n{res_str}\n\n"

### Ітерації обчислення, перевірка оптимальності

In [3]:
step1 = LpInvMtxStep(A=mtx_vars_all,
                     C=C,
                     b=b,
                     Cb = Cb)
print(step1)

Iter #:
1

Values amount:
7

Values dict:
{0: 'X_1', 1: 'X_2', 2: 'X_3', 3: 'X_4', 4: 'X_5', 5: 'X_6', 6: 'X_7'}

Matrix A:
[[ 2  1 -1  0]
 [ 1 -1  0  0]
 [-1  2  0 -1]]

C:
[-6 -4  0  0]

b:
[3 1 1]

B:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Basis index:
[4 5 6]

Basis varnames:
X_5, X_6, X_7

B_inv:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

CbB_inv:
[-10000.      0. -10000.]

BinvA:
[[ 2.  1. -1.  0.]
 [ 1. -1.  0.  0.]
 [-1.  2.  0. -1.]]

Binv_b:
[3. 1. 1.]

CbB_inv_b:
-40000.0

CbBinvA_minus_C:
[ -9994. -29996.  10000.  10000.]

min_elem_order:
X2 = -29996.0

ratio:
[ 3.  -1.   0.5]

min ratio value:
0.5

leave:
X_7

New Cb:
[-10000      0     -4]

New B:
[[ 1.  0.  1.]
 [ 0.  1. -1.]
 [ 0.  0.  2.]]

Result:
PLANE STILL CONSIST NEGATIVE VALUE(S)
------------------------------




In [4]:
step1.get_next_step()

{'A': array([[ 2,  1, -1,  0],
        [ 1, -1,  0,  0],
        [-1,  2,  0, -1]]),
 'C': array([-6, -4,  0,  0]),
 'b': array([3, 1, 1]),
 'B': array([[ 1.,  0.,  1.],
        [ 0.,  1., -1.],
        [ 0.,  0.,  2.]]),
 'Cb': array([-10000,      0,     -4]),
 'current_basis_indexes': array([4, 5, 1]),
 'step': 2}

In [5]:
step2 = LpInvMtxStep(**step1.get_next_step())
print(step2)

Iter #:
2

Values amount:
7

Values dict:
{0: 'X_1', 1: 'X_2', 2: 'X_3', 3: 'X_4', 4: 'X_5', 5: 'X_6', 6: 'X_7'}

Matrix A:
[[ 2  1 -1  0]
 [ 1 -1  0  0]
 [-1  2  0 -1]]

C:
[-6 -4  0  0]

b:
[3 1 1]

B:
[[ 1.  0.  1.]
 [ 0.  1. -1.]
 [ 0.  0.  2.]]

Basis index:
[4 5 1]

Basis varnames:
X_5, X_6, X_2

B_inv:
[[ 1.   0.  -0.5]
 [ 0.   1.   0.5]
 [ 0.   0.   0.5]]

CbB_inv:
[-10000.      0.   4998.]

BinvA:
[[ 2.5  0.  -1.   0.5]
 [ 0.5  0.   0.  -0.5]
 [-0.5  1.   0.  -0.5]]

Binv_b:
[2.5 1.5 0.5]

CbB_inv_b:
-25002.0

CbBinvA_minus_C:
[-24992.      0.  10000.  -4998.]

min_elem_order:
X1 = -24992.0

ratio:
[ 1.  3. -1.]

min ratio value:
1.0

leave:
X_5

New Cb:
[-6  0 -4]

New B:
[[ 2.  0.  1.]
 [ 1.  1. -1.]
 [-1.  0.  2.]]

Result:
PLANE STILL CONSIST NEGATIVE VALUE(S)
------------------------------




In [6]:
step2.get_next_step()

{'A': array([[ 2,  1, -1,  0],
        [ 1, -1,  0,  0],
        [-1,  2,  0, -1]]),
 'C': array([-6, -4,  0,  0]),
 'b': array([3, 1, 1]),
 'B': array([[ 2.,  0.,  1.],
        [ 1.,  1., -1.],
        [-1.,  0.,  2.]]),
 'Cb': array([-6,  0, -4]),
 'current_basis_indexes': array([0, 5, 1]),
 'step': 3}

In [7]:
step3 = LpInvMtxStep(**step2.get_next_step())
print(step3)

Iter #:
3

Values amount:
7

Values dict:
{0: 'X_1', 1: 'X_2', 2: 'X_3', 3: 'X_4', 4: 'X_5', 5: 'X_6', 6: 'X_7'}

Matrix A:
[[ 2  1 -1  0]
 [ 1 -1  0  0]
 [-1  2  0 -1]]

C:
[-6 -4  0  0]

b:
[3 1 1]

B:
[[ 2.  0.  1.]
 [ 1.  1. -1.]
 [-1.  0.  2.]]

Basis index:
[0 5 1]

Basis varnames:
X_1, X_6, X_2

B_inv:
[[ 0.4  0.  -0.2]
 [-0.2  1.   0.6]
 [ 0.2  0.   0.4]]

CbB_inv:
[-3.2  0.  -0.4]

BinvA:
[[ 1.00000000e+00  0.00000000e+00 -4.00000000e-01  2.00000000e-01]
 [ 0.00000000e+00  2.22044605e-16  2.00000000e-01 -6.00000000e-01]
 [ 0.00000000e+00  1.00000000e+00 -2.00000000e-01 -4.00000000e-01]]

Binv_b:
[1. 1. 1.]

CbB_inv_b:
-10.000000000000002

CbBinvA_minus_C:
[0.  0.  3.2 0.4]

min_elem_order:
X1 = 0.0

ratio:
[ 1. inf inf]

min ratio value:
1.0000000000000002

leave:
X_1

New Cb:
None

New B:
None

Result:
------------------------------
ITERATING STOPPED: 
BEST FUNC: -10.000000000000002 
VARS:      
X_1: 1.0000000000000002
X_6: 1.0000000000000002
X_2: 1.0

-----------------------

In [8]:
step3.get_next_step()

{'A': array([[ 2,  1, -1,  0],
        [ 1, -1,  0,  0],
        [-1,  2,  0, -1]]),
 'C': array([-6, -4,  0,  0]),
 'b': array([3, 1, 1]),
 'B': None,
 'Cb': None,
 'current_basis_indexes': None,
 'step': 4}

### Остання ітерація

In [9]:
step4 = LpInvMtxStep(**step3.get_next_step())
print(step4)

Iter #:
4

Values amount:
7

Values dict:
{0: 'X_1', 1: 'X_2', 2: 'X_3', 3: 'X_4', 4: 'X_5', 5: 'X_6', 6: 'X_7'}

Matrix A:
[[ 2  1 -1  0]
 [ 1 -1  0  0]
 [-1  2  0 -1]]

C:
[-6 -4  0  0]

b:
[3 1 1]

B:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Basis index:
[4 5 6]

Basis varnames:
X_5, X_6, X_7

B_inv:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

CbB_inv:
[0. 0. 0.]

BinvA:
[[ 2.  1. -1.  0.]
 [ 1. -1.  0.  0.]
 [-1.  2.  0. -1.]]

Binv_b:
[3. 1. 1.]

CbB_inv_b:
0.0

CbBinvA_minus_C:
[6. 4. 0. 0.]

min_elem_order:
X3 = 0.0

ratio:
[-3. inf inf]

min ratio value:
inf

leave:
X_6

New Cb:
None

New B:
None

Result:
------------------------------
ITERATING STOPPED: 
BEST FUNC: 0.0 
VARS:      
X_5: 3.0
X_6: 1.0
X_7: 1.0

------------------------------




### Виконання у циклі (до досягнення оптимальності)

In [10]:
# Клітинка, що рахувати не покроково:
res = False

initial_step = LpInvMtxStep(A=mtx_vars_all,
                            C=C,
                            b=b,
                            Cb = Cb)

next_step_data = initial_step.get_next_step()

while not res:
    new_step = LpInvMtxStep(**next_step_data)
    res = new_step.get_result()['stop_status']
    next_step_data = new_step.get_next_step()

print(new_step) # інформація про ітерацію на який досягнуто оптимального плану


Iter #:
3

Values amount:
7

Values dict:
{0: 'X_1', 1: 'X_2', 2: 'X_3', 3: 'X_4', 4: 'X_5', 5: 'X_6', 6: 'X_7'}

Matrix A:
[[ 2  1 -1  0]
 [ 1 -1  0  0]
 [-1  2  0 -1]]

C:
[-6 -4  0  0]

b:
[3 1 1]

B:
[[ 2.  0.  1.]
 [ 1.  1. -1.]
 [-1.  0.  2.]]

Basis index:
[0 5 1]

Basis varnames:
X_1, X_6, X_2

B_inv:
[[ 0.4  0.  -0.2]
 [-0.2  1.   0.6]
 [ 0.2  0.   0.4]]

CbB_inv:
[-3.2  0.  -0.4]

BinvA:
[[ 1.00000000e+00  0.00000000e+00 -4.00000000e-01  2.00000000e-01]
 [ 0.00000000e+00  2.22044605e-16  2.00000000e-01 -6.00000000e-01]
 [ 0.00000000e+00  1.00000000e+00 -2.00000000e-01 -4.00000000e-01]]

Binv_b:
[1. 1. 1.]

CbB_inv_b:
-10.000000000000002

CbBinvA_minus_C:
[0.  0.  3.2 0.4]

min_elem_order:
X1 = 0.0

ratio:
[ 1. inf inf]

min ratio value:
1.0000000000000002

leave:
X_1

New Cb:
None

New B:
None

Result:
------------------------------
ITERATING STOPPED: 
BEST FUNC: -10.000000000000002 
VARS:      
X_1: 1.0000000000000002
X_6: 1.0000000000000002
X_2: 1.0

-----------------------

### Для задачі мінімізації (зважаючи на обернення знаку функції) відповідь: F_min = 10 (x_1 = 1, x_2 = 1)

### Перевірка за допомогою моделювання PuLP

In [11]:
from pulp import *
import numpy as np

# Задача лінійного програмування з мінімізації ц.ф.
problem = LpProblem("Minimization Problem", LpMinimize)

# Змінні рішення x1 і x2
x1 = LpVariable("x1", lowBound=0)
x2 = LpVariable("x2", lowBound=0)

# Ц.ф.
objective_function = 6 * x1 + 4 * x2
problem += objective_function

# Обмеження
problem += 2 * x1 + x2 >= 3
problem += x1 - x2 <= 1
problem += -x1 + 2 * x2 >= 1


# Розв'язуємо задачу лінійного програмування
problem.solve()

# Виводимо статус розв'язку
print("Status:", LpStatus[problem.status])

# Виводимо оптимальне значення змінних і значення цільової функції
print("Optimal Solution:")
print("x1 =", value(x1))
print("x2 =", value(x2))

print("F_min =", value(problem.objective))

Status: Optimal
Optimal Solution:
x1 = 1.0
x2 = 1.0
F_min = 10.0


C:\Users\matematik\AppData\Local\Programs\Python\Python310\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
